In [ ]:
'''
features_scala_prezzo = [
    #"Close",
    # "EMA_5", 
    "EMA_20", 
    "EMA_50",
    "EMA_100",
    #"Open",  
    #"High",
    #"Low",
    # "PSAR",
    # "SUPERT", 
]

features_da_scalare_singolarmente = [
    "Volume",
    # "ATR",
    # "PSARaf",
    # "ADX",
    # "OBV"
]

features_oscillatori = [
    "MACDh",    
    # "MACD",
    # "MACDs",
    "AROONOSC",
    "TRIX",
    # "TRIXs",
    "DM_OSC",
    "TSI",
    # "TSIs",
    # "ROC_10",
    "KVO",
    # "KVOs",
    # "VI_OSC"
]

features_no_scala = [
    # "SUPERTd",  
    # "PSARr",
    # "CMF",
    # "VHF",
    # "VTX_OSC"
]

features_candele = [
    #"CDL_2CROWS", "CDL_3BLACKCROWS", "CDL_3INSIDE", "CDL_3LINESTRIKE", "CDL_3OUTSIDE", "CDL_3STARSINSOUTH", "CDL_3WHITESOLDIERS", "CDL_ABANDONEDBABY", "CDL_ADVANCEBLOCK", "CDL_BELTHOLD", "CDL_BREAKAWAY", "CDL_CLOSINGMARUBOZU", "CDL_CONCEALBABYSWALL", "CDL_COUNTERATTACK", "CDL_DARKCLOUDCOVER", "CDL_DOJI_10_0.1", "CDL_DOJISTAR", "CDL_DRAGONFLYDOJI", "CDL_ENGULFING", "CDL_EVENINGDOJISTAR", "CDL_EVENINGSTAR", "CDL_GAPSIDESIDEWHITE", "CDL_GRAVESTONEDOJI", "CDL_HAMMER", "CDL_HANGINGMAN", "CDL_HARAMI", "CDL_HARAMICROSS", "CDL_HIGHWAVE", "CDL_HIKKAKE", "CDL_HIKKAKEMOD", "CDL_HOMINGPIGEON", "CDL_IDENTICAL3CROWS", "CDL_INNECK", "CDL_INSIDE", "CDL_INVERTEDHAMMER", "CDL_KICKING", "CDL_KICKINGBYLENGTH", "CDL_LADDERBOTTOM", "CDL_LONGLEGGEDDOJI", "CDL_LONGLINE", "CDL_MARUBOZU", "CDL_MATCHINGLOW", "CDL_MATHOLD", "CDL_MORNINGDOJISTAR", "CDL_MORNINGSTAR", "CDL_ONNECK", "CDL_PIERCING", "CDL_RICKSHAWMAN", "CDL_RISEFALL3METHODS", "CDL_SEPARATINGLINES", "CDL_SHOOTINGSTAR", "CDL_SHORTLINE", "CDL_SPINNINGTOP", "CDL_STALLEDPATTERN", "CDL_STICKSANDWICH", "CDL_TAKURI", "CDL_TASUKIGAP", "CDL_THRUSTING", "CDL_TRISTAR", "CDL_UNIQUE3RIVER", "CDL_UPSIDEGAP2CROWS", "CDL_XSIDEGAP3METHODS",
]

targets = [
#    "EMA_5",
#    "EMA_20", 
#    "EMA_50",
    #"Open",
    #"High",
    #"Low",
    "Target"
]
'''

In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
from multiprocessing import Pool, cpu_count, Manager, Value
import tensorflow as tf
from tensorflow_addons.metrics import F1Score
from tensorflow.keras.models import load_model, Sequential
from tensorflow.keras.layers import Bidirectional, BatchNormalization, LSTM, Dropout, Dense, Conv1D, Flatten
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, LearningRateScheduler
from tensorflow.keras import regularizers
from tensorflow.keras.metrics import Precision, Recall
import tensorflow_addons as tfa
from datetime import timedelta
import json
import os
from functools import partial
from sklearn.preprocessing import MinMaxScaler
from imblearn.under_sampling import RandomUnderSampler
import pandas_ta as ta

_features_scala_prezzo_tutte = [
    "Close",
    "EMA_5", 
    "EMA_20", 
    "EMA_50",
    "EMA_100",
    "Open",  
    "High",
    "Low",
    "PSAR",
    "SUPERT", 
]

_features_da_scalare_singolarmente_tutte = [
    "Volume",
    "ATR",
    "PSARaf",
    "ADX",
    "OBV"
]

_features_oscillatori_tutte = [
    "MACDh",    
    "MACD",
    "MACDs",
    "AROONOSC",
    "TRIX",
    "TRIXs",
    "DM_OSC",
    "TSI",
    "TSIs",
    "ROC_10",
    "KVO",
    "KVOs",
    "VI_OSC"
]

_features_no_scala_tutte = [
    "SUPERTd",  
    "PSARr",
    "CMF",
    "VHF",
    "VTX_OSC"
]

_features_candele_tutte = [
    "CDL_2CROWS", "CDL_3BLACKCROWS", "CDL_3INSIDE", "CDL_3LINESTRIKE", "CDL_3OUTSIDE", "CDL_3STARSINSOUTH", "CDL_3WHITESOLDIERS", "CDL_ABANDONEDBABY", "CDL_ADVANCEBLOCK", "CDL_BELTHOLD", "CDL_BREAKAWAY", "CDL_CLOSINGMARUBOZU", "CDL_CONCEALBABYSWALL", "CDL_COUNTERATTACK", "CDL_DARKCLOUDCOVER", "CDL_DOJI_10_0.1", "CDL_DOJISTAR", "CDL_DRAGONFLYDOJI", "CDL_ENGULFING", "CDL_EVENINGDOJISTAR", "CDL_EVENINGSTAR", "CDL_GAPSIDESIDEWHITE", "CDL_GRAVESTONEDOJI", "CDL_HAMMER", "CDL_HANGINGMAN", "CDL_HARAMI", "CDL_HARAMICROSS", "CDL_HIGHWAVE", "CDL_HIKKAKE", "CDL_HIKKAKEMOD", "CDL_HOMINGPIGEON", "CDL_IDENTICAL3CROWS", "CDL_INNECK", "CDL_INSIDE", "CDL_INVERTEDHAMMER", "CDL_KICKING", "CDL_KICKINGBYLENGTH", "CDL_LADDERBOTTOM", "CDL_LONGLEGGEDDOJI", "CDL_LONGLINE", "CDL_MARUBOZU", "CDL_MATCHINGLOW", "CDL_MATHOLD", "CDL_MORNINGDOJISTAR", "CDL_MORNINGSTAR", "CDL_ONNECK", "CDL_PIERCING", "CDL_RICKSHAWMAN", "CDL_RISEFALL3METHODS", "CDL_SEPARATINGLINES", "CDL_SHOOTINGSTAR", "CDL_SHORTLINE", "CDL_SPINNINGTOP", "CDL_STALLEDPATTERN", "CDL_STICKSANDWICH", "CDL_TAKURI", "CDL_TASUKIGAP", "CDL_THRUSTING", "CDL_TRISTAR", "CDL_UNIQUE3RIVER", "CDL_UPSIDEGAP2CROWS", "CDL_XSIDEGAP3METHODS",
]

def pct_change(valore_iniziale, valore_finale):
    try:
        return ((valore_finale - valore_iniziale) / valore_iniziale) * 100
    except ZeroDivisionError:
        return 0

def analizza_ticker(nome_simbolo, start, end, progress=True, dropna_iniziali=False, dropna_finali=False):
    start_str = start.strftime('%Y-%m-%d')
    end_str = end.strftime('%Y-%m-%d')
    df = yf.download(nome_simbolo, start=start_str, end=end_str, progress=progress)
    df.index = pd.to_datetime(df.index)
    df = crea_indicatori(df)
    if dropna_iniziali:
        idx = df[df.notna().all(axis=1) == True].index[0]
        df = df[idx:]
    if dropna_finali:
        idx = df[df.notna().all(axis=1) == True].index[-1]
        df = df[:idx]
    df = imposta_target(df)
    return df

def dropna_iniziali(df):
    idx = df[df.notna().all(axis=1) == True].index[0]
    df = df[idx:]
    return df

def dropna_finali(df):
    idx = df[df.notna().all(axis=1) == True].index[-1]
    df = df[:idx]
    return df

def imposta_target(df):
    df['Target'] = (
        (df['EMA_20_5d'] > df['EMA_20']) & (df['EMA_20_10d'] > df['EMA_20']) & (df['EMA_20_15d'] > df['EMA_20'])  & (df['EMA_20_20d'] > df['EMA_20']) &
        (df['EMA_20_5d'] > df['EMA_50_5d']) &
        (df['EMA_20_10d'] > df['EMA_50_10d']) &
        (df['EMA_20_15d'] > df['EMA_50_15d']) &
        (df['EMA_20_20d'] > df['EMA_50_20d'])
    )
    return df
    
def crea_indicatori(df):
    def __trova_massimi_minimi(df, periodo):
        mezzo_periodo = periodo // 2

        massimi_passati = df['Close'].shift(1).rolling(mezzo_periodo).max()
        massimi_futuri = df['Close'][::-1].shift(1).rolling(mezzo_periodo).max()[::-1]
        idx_massimi = (df["Close"] >= massimi_passati) & (df["Close"] >= massimi_futuri)
        df.loc[idx_massimi, "MaxMinRel"] = periodo

        minimi_passati = df['Close'].shift(1).rolling(mezzo_periodo).min()
        minimi_futuri = df['Close'][::-1].shift(1).rolling(mezzo_periodo).min()[::-1]
        idx_minimi = (df["Close"] <= minimi_passati) & (df["Close"] <= minimi_futuri)
        df.loc[idx_minimi, "MaxMinRel"] = -periodo
            
        return df

    def __rinomina_colonne(df):
        df = df.rename(columns={
            'PSARaf_0.02_0.2': 'PSARaf',
            'PSARr_0.02_0.2': 'PSARr',
            'MACD_20_50_9': 'MACD',
            'MACDh_20_50_9': 'MACDh',
            'MACDs_20_50_9': 'MACDs',
            'TSI_13_25_13': 'TSI',
            'TSIs_13_25_13': 'TSIs',
            'SUPERT_20_3.0': 'SUPERT',
            'SUPERTd_20_3.0': 'SUPERTd',
            'ADX_20': 'ADX',
            'DMP_20': 'DMP',
            'DMN_20': 'DMN',
            'CMF_10': 'CMF',
            'TRIX_18_9': 'TRIX',
            'TRIXs_18_9': 'TRIXs',
            'KVO_34_55_13': 'KVO',
            'KVOs_34_55_13': 'KVOs',
            'DCL_20_20': 'DCL',
            'DCM_20_20': 'DCM',
            'DCU_20_20': 'DCU',
            'VTXP_20': 'VTXP',
            'VTXM_20': 'VTXM',
            'AROOND_20': 'AROOND',
            'AROONU_20': 'AROONU',
            'AROONOSC_20': 'AROONOSC',
            'NVI_1': 'NVI',
            'PVI_1': 'PVI',
            'VHF_20': 'VHF',
            'ATRr_14': 'ATR'
        })
        return df

    def __calcolo_drawdown_gain(df, periodo):
        df[f"Max_High_Futuro_{periodo}d"] = df["High"].shift(-periodo).rolling(periodo).max()
        df[f"Min_Low_Futuro_{periodo}d"] = df["Low"].shift(-periodo).rolling(periodo).min()
        df[f"Drawdown_{periodo}d"] = df["Open"] - df[f"Min_Low_Futuro_{periodo}d"]
        df[f"Drawdown_{periodo}d"] = df[f"Drawdown_{periodo}d"].where(df[f"Drawdown_{periodo}d"] > 0, 0)
        df[f"Perc_Max_High_Futuro_{periodo}d"] = ((df[f"Max_High_Futuro_{periodo}d"] - df["Open"]) / df["Open"]) * 100
        df[f"Perc_Drawdown_{periodo}d"] = ((df[f"Drawdown_{periodo}d"]) / df["Open"]) * 100 
        df[f"Perc_Drawdown_{periodo}d"] = df[f"Perc_Drawdown_{periodo}d"].where(df[f"Perc_Drawdown_{periodo}d"] > 0, 0)
        df.drop(columns=[f"Max_High_Futuro_{periodo}d", f"Min_Low_Futuro_{periodo}d", f"Drawdown_{periodo}d"], axis=1, inplace=True)
        return df
    
    psar = ta.psar(high=df["High"], low=df["Low"], close=df["Close"], af0=0.02, af=0.02, max_af=0.2)
    psar["PSAR"] = psar["PSARl_0.02_0.2"].combine_first(psar["PSARs_0.02_0.2"])
    psar.drop(["PSARl_0.02_0.2", "PSARs_0.02_0.2"], axis=1, inplace=True)
    macd = ta.macd(close=df["Close"], fast=20, slow=50, signal=9)
    tsi = ta.tsi(close=df["Close"], fast=13, slow=25)
    supertrend = ta.supertrend(high=df["High"], low=df["Low"], close=df["Close"], length=20, multiplier=3)
    supertrend.drop(["SUPERTl_20_3.0", "SUPERTs_20_3.0"], axis=1, inplace=True)
    ema5 = ta.ema(close=df["Close"], length=5)
    ema20 = ta.ema(close=df["Close"], length=20)
    ema50 = ta.ema(close=df["Close"], length=50)
    ema100 = ta.ema(close=df["Close"], length=100)
    adx = ta.adx(high=df["High"], low=df["Low"], close=df["Close"], length=20)
    roc = ta.roc(close=df["Close"], length=10)
    cmf = ta.cmf(high=df["High"], low=df["Low"], close=df["Close"], volume=df['Volume'], length=10)
    trix = ta.trix(close=df['Close'], length=18)
    klinger = ta.kvo(high=df["High"], low=df["Low"], close=df["Close"], volume=df['Volume'], short=34, long=55)
    vi = ta.vortex(high=df['High'], low=df['Low'], close=df['Close'], length=20)
    aroon = ta.aroon(high=df['High'], low=df['Low'], close=df['Close'], length=20)
    nvi = ta.nvi(close=df['Close'], volume=df['Volume'])
    pvi = ta.pvi(close=df['Close'], volume=df['Volume'])
    vhf = ta.vhf(close=df['Close'], length=20)
    atr = ta.atr(high=df['High'], low=df['Low'], close=df['Close'])
    obv = ta.obv(close=df["Close"], volume=df["Volume"])
    #candele = ta.cdl_pattern(open_=df["Open"], high=df["High"], low=df["Low"], close=df["Close"])

    df = pd.concat([df, ema5, ema20, ema50, ema100, psar, macd, tsi, supertrend, adx, trix, vi, aroon, nvi, pvi, atr, cmf, roc, klinger, vhf, obv], axis=1)

    df = __rinomina_colonne(df)

    df = __calcolo_drawdown_gain(df, 20)
    #df = __calcolo_drawdown_gain(df, 50)
    #df = __calcolo_drawdown_gain(df, 100)
    #df["max_gain"] = df[["Perc_Max_High_Futuro_20d", "Perc_Max_High_Futuro_50d", "Perc_Max_High_Futuro_100d"]].max(axis=1)
    #df["max_drawdown"] = df[["Perc_Drawdown_20d", "Perc_Drawdown_50d", "Perc_Drawdown_100d"]].min(axis=1)

    df['EMA_20_5d'] = df['EMA_20'].shift(-5)
    df['EMA_20_10d'] = df['EMA_20'].shift(-10)
    df['EMA_20_15d'] = df['EMA_20'].shift(-15)
    df['EMA_20_20d'] = df['EMA_20'].shift(-20)
    df['EMA_50_5d'] = df['EMA_50'].shift(-5)
    df['EMA_50_10d'] = df['EMA_50'].shift(-10)
    df['EMA_50_15d'] = df['EMA_50'].shift(-15)
    df['EMA_50_20d'] = df['EMA_50'].shift(-20)
    df['Close_5d'] = df['Close'].shift(-5)
    df['Close_10d'] = df['Close'].shift(-10)
    df['Close_15d'] = df['Close'].shift(-15)
    df['Close_20d'] = df['Close'].shift(-20)
    df['EMA_5_5d'] = df['EMA_5'].shift(-5)
    df['EMA_5_10d'] = df['EMA_5'].shift(-10)
    df['EMA_5_15d'] = df['EMA_5'].shift(-15)
    df['EMA_5_20d'] = df['EMA_5'].shift(-20)
    #df['Close_1d'] = df['Close'].shift(-1)
    #df['perc_EMA_5_20d'] = ((df['EMA_5_20d'] - df['EMA_5']) / df['EMA_5']) * 100
    #df['perc_Close_20d'] = ((df['Close_20d'] - df['Close']) / df['Close']) * 100
    #df['incrocio_verde_gialla'] = (ta.cross(df['EMA_20'], df['EMA_50'], above=True)).astype("int8")
    #df["incrocio_passato_verde_gialla_10d"] = df["incrocio_verde_gialla"].rolling(10).sum()
    
    #df['HLC3'] = ((df['High'] + df['Low'] + df['Close']) / 3)
    df["DM_OSC"] = (df["DMP"] - df["DMN"])
    df["VTX_OSC"] = (df["VTXP"] - df["VTXM"])
    df["VI_OSC"] = (df["PVI"] - df["NVI"])
    
    df.drop(columns=["DMP", "DMN", "VTXP", "VTXM", "PVI", "NVI", "AROOND", "AROONU"], inplace=True, axis=1)
    
    #df["MaxMinRel"] = 0
    #df = __trova_massimi_minimi(df, 20)   
    #df = __trova_massimi_minimi(df, 50)   
    #df = __trova_massimi_minimi(df, 100)         
    
    # (
    #     (df["Perc_Drawdown_20d"] < 5) & 
    #     (df['Close_5d'] > df['Close']) & (df['Close_10d'] > df['Close']) & (df['Close_15d'] > df['Close'])  & (df['Close_20d'] > df['Close']) &
    #     (df['Close_5d'] > df['EMA_20_5d']) & (df['EMA_20_5d'] > df['EMA_50_5d']) &
    #     (df['Close_10d'] > df['EMA_20_10d']) & (df['EMA_20_10d'] > df['EMA_50_10d']) &
    #     (df['Close_15d'] > df['EMA_20_15d']) & (df['EMA_20_15d'] > df['EMA_50_15d']) &
    #     (df['Close_20d'] > df['EMA_20_20d']) & (df['EMA_20_20d'] > df['EMA_50_20d']) &
    #     (df['EMA_20'] < df['EMA_50'])
    # )

    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    return df

def _scarica(nome_simbolo, scarica_prima, scarica_dopo, data_inizio, data_fine, append):
    try:
        if append == True:
            ticker = pd.read_hdf(f'tickers/{nome_simbolo}.h5', "ticker")
            ti_min = ticker.index.min()
            ti_max = ticker.index.max()
            if scarica_prima:
                inizio = data_inizio - pd.Timedelta(days=365)
                fine = ti_min - pd.Timedelta(days=1)
                df_inizio = analizza_ticker(nome_simbolo, start=inizio, end=fine, progress=False, dropna_iniziali=True, dropna_finali=False)
                ticker = pd.concat([df_inizio, ticker], axis=0, ignore_index=False)
            if scarica_dopo:
                inizio = ti_max - pd.Timedelta(days=365) 
                fine = data_fine
                df_fine = analizza_ticker(nome_simbolo, start=inizio, end=fine, progress=False, dropna_iniziali=True, dropna_finali=False)
                ticker = ticker[ticker.index < df_fine.index.min()]
                ticker = pd.concat([ticker, df_fine], axis=0, ignore_index=False)
        else:
            ticker = analizza_ticker(nome_simbolo, start=data_inizio, end=data_fine, progress=False, dropna_iniziali=True, dropna_finali=False)
        return nome_simbolo, ticker, ""
    except Exception as e:
        return nome_simbolo, None, str(e)

def _callback_tickers(result, totale_processati, tot_tickers):
    nome_simbolo, ticker, error = result
    if error == "":
        ticker.to_hdf(f'tickers/{nome_simbolo}.h5', key='ticker', mode='w')
    else:
        print(f"Errore su funzione di callback per {nome_simbolo}: {error}")

    with totale_processati.get_lock(): 
        totale_processati.value += 1
    print(f"{totale_processati.value}/{tot_tickers}) Completato scarico dati ticker {nome_simbolo}")   

def _carica_screener(nome_simbolo, lista_scr, prob):
    try:
        df = pd.read_hdf(f'screeners/{nome_simbolo}.h5', 'screener')
        df.index.set_names(['Data'], inplace=True)
        df['Ticker'] = nome_simbolo
        df.set_index('Ticker', append=True, inplace=True)
        df = df[df['Previsione'] > prob]
        lista_scr.append(df)
        return nome_simbolo, ""
    except Exception as e:
        return nome_simbolo, str(e)

def _carica_screener_callback(result, totale_processati, tot_tickers):
    nome_simbolo, error = result
    if error != "":
        print(f"Errore su funzione di callback per {nome_simbolo}: {error}")

    with totale_processati.get_lock(): 
        totale_processati.value += 1
    print(f"{totale_processati.value}/{tot_tickers}) Caricato su screener ticker {nome_simbolo}")   

class Posizione:
    def __init__(self, simbolo, ticker, data, n_azioni, bilancio, max_giorni_apertura, stop_loss, take_profit) -> None:
        self.simbolo = simbolo
        self.ticker = ticker
        self.data = data
        self.prezzo_unitario = ticker['Open'].iloc[0]
        self.n_azioni = n_azioni
        self.stop_loss = self.prezzo_unitario * (1 - stop_loss)
        self.take_profit = self.prezzo_unitario * (1 + take_profit)
        self.prezzo_tot = self.n_azioni * self.prezzo_unitario
        self.bilancio = bilancio - self.prezzo_tot 
        self.giorni_apertura = 1
        self.max_giorni_apertura = max_giorni_apertura
        self.valore = self.prezzo_tot
        self._colonne = ['Simbolo', 'Data', 'Tipo', 'Prezzo_unitario', 'n_azioni', 'Prezzo_tot', 'SL', 'TP', 'Bilancio', 'Valore_posizione', 'Giorni_apertura', 'Esito', 'Vincita', 'Perc']
        
    def to_df(self) -> pd.DataFrame:
        return pd.DataFrame([
            [self.simbolo, self.data.normalize(), 'COMPRA', self.prezzo_unitario.round(2), int(self.n_azioni), self.prezzo_tot.round(2), self.stop_loss.round(2), self.take_profit.round(2), self.bilancio.round(2), self.valore.round(2), int(self.giorni_apertura), '', 0, 0]
        ], columns=self._colonne)

    def chiudi(self, data, bilancio, forza_chiusura=False):
        if (data in self.ticker.index) or (forza_chiusura):
            if forza_chiusura:
                dati_attuali = self.ticker.iloc[-1:]
            else:
                dati_attuali = self.ticker[self.ticker.index == data]
            self.giorni_apertura += 1
            prezzo_attuale = dati_attuali['Open'].iloc[0]
            prezzo_tot = prezzo_attuale * self.n_azioni
            self.valore = prezzo_tot
            if (self.giorni_apertura > self.max_giorni_apertura) or (prezzo_attuale < self.stop_loss) or (prezzo_attuale > self.take_profit) or (forza_chiusura):
                bilancio += prezzo_tot
                if prezzo_attuale > self.prezzo_unitario:
                    esito = 'VINCITA'
                else:
                    esito = 'PERDITA'
                esito_tot = prezzo_tot - self.prezzo_tot
                esito_pct = pct_change(self.prezzo_unitario, prezzo_attuale)
                return pd.DataFrame([
                    [self.simbolo, data.normalize(), 'VENDI', prezzo_attuale.round(2), int(self.n_azioni), prezzo_tot.round(2), self.stop_loss.round(2), self.take_profit.round(2), bilancio.round(2), self.valore.round(2), int(self.giorni_apertura), esito, esito_tot.round(2), int(esito_pct)]
                ], columns=self._colonne)
        return None

class Borsa:
    def __init__(self, n_simboli_contemporanei, bilancio_iniziale, probabilità_per_acquisto, stop_loss, take_profit, giorni_max_posizione, data_inizio=pd.Timestamp(year=2005, month=1, day=1), data_fine=pd.Timestamp.now().normalize()):
        self.inizializza_gpu()

        self.N_SIMBOLI = n_simboli_contemporanei

        self.DATA_INIZIO = pd.to_datetime(data_inizio)
        while self.DATA_INIZIO.dayofweek >= 5:
            self.DATA_INIZIO += pd.Timedelta(days=1)

        self.DATA_FINE = pd.to_datetime(data_fine)
        while self.DATA_FINE.dayofweek >= 5:
            self.DATA_FINE -= pd.Timedelta(days=1)

        self.BILANCIO_INIZIALE = bilancio_iniziale
        self.PROBABILITA_PER_ACQUISTO = probabilità_per_acquisto
        self.SL = stop_loss
        self.TP = take_profit
        self.GIORNI_POS = giorni_max_posizione
        self._posizioni = []
        self._valore_posizioni = 0
        self._bilancio = None
        self._data_corrente = None
        self._bilancio_per_simbolo = None
        self.esito_trading = None

        self.modello = Modello('primo_modello')
        
        self.lista_tickers = pd.read_parquet("lista_ticker.parquet")
        self.tot_tickers = len(self.lista_tickers)
        self.screener = pd.DataFrame()

        try:
            if os.path.exists(f'tickers/_indice.json'):
                with open(f'tickers/_indice.json', 'r') as jsonfile:
                    indice = json.load(jsonfile)
                prima_data = pd.to_datetime(indice['prima_data'])
                ultima_data = pd.to_datetime(indice['ultima_data'])
                
                if (self.DATA_INIZIO < prima_data):
                    scarica_prima = True
                else:
                    scarica_prima = False

                if (self.DATA_FINE > ultima_data):
                    scarica_dopo = True
                else:
                    scarica_dopo = False

                if scarica_prima or scarica_dopo:
                    self.scarica_tickers(scarica_prima, scarica_dopo, self.DATA_INIZIO, self.DATA_FINE, append=True)
                    self.avvia_screener(append=True)    
                    self.carica_screener()
                    self.screener.to_pickle('screeners/_screener.pickle')
                else:
                    self.screener = pd.read_pickle('screeners/_screener.pickle')
            else:
                self.scarica_tickers(scarica_prima=True, scarica_dopo=True, data_inizio=self.DATA_INIZIO, data_fine=self.DATA_FINE, append=False)
                self.avvia_screener(append=False)    
                self.carica_screener()
                self.screener.to_pickle('screeners/_screener.pickle')
        except Exception as e:
            print(str(e))
        

    def inizializza_gpu(self):
        # Ottiene la lista di tutte le GPU fisiche disponibili
        gpus = tf.config.experimental.list_physical_devices('GPU')
        if gpus:
            try:
                # Imposta TensorFlow per utilizzare tutte le GPU visibili
                tf.config.experimental.set_visible_devices(gpus, 'GPU')
                for gpu in gpus:
                    # Imposta TensorFlow per allocare dinamicamente la memoria su ogni GPU disponibile
                    tf.config.experimental.set_memory_growth(gpu, True)
            except RuntimeError as e:
                # L'eccezione viene sollevata se si tenta di impostare la visibilità delle GPU o la crescita della memoria
                # dopo che la sessione di TensorFlow è stata inizializzata
                print(e)

    def carica_screener(self):
        manager = Manager()
        lista_scr = manager.list()
        totale_processati = Value('i', 0)
        with Pool(cpu_count()) as p:
            callback_with_args = partial(_carica_screener_callback, totale_processati=totale_processati, tot_tickers=self.tot_tickers)
            for i in range(0, self.tot_tickers):
                nome_simbolo = self.lista_tickers.iloc[i]["Ticker"]
                param = (nome_simbolo, lista_scr, self.PROBABILITA_PER_ACQUISTO)
                p.apply_async(_carica_screener, args=param, callback=callback_with_args)
            p.close()
            p.join()  
        self.screener = pd.concat(lista_scr, axis=0, ignore_index=False)
        self.screener = self.screener.sort_values(by=['Data', 'Previsione'], ascending=[True, False])

    def scarica_tickers(self, scarica_prima=True, scarica_dopo=True, data_inizio=pd.Timestamp(year=2005, month=1, day=1), data_fine=pd.Timestamp.now().normalize(), append=False) -> None: 
        tot_tickers = len(self.lista_tickers)
        totale_processati = Value('i', 0)
        with Pool(cpu_count()) as p:
            callback_with_args = partial(_callback_tickers, totale_processati=totale_processati, tot_tickers=tot_tickers)
            for i in range(0, tot_tickers):
                nome_simbolo = self.lista_tickers.iloc[i]["Ticker"]
                param = (nome_simbolo, scarica_prima, scarica_dopo, data_inizio, data_fine, append)
                p.apply_async(_scarica, args=param, callback=callback_with_args)
            p.close()
            p.join()     

        indice = {
            'prima_data': self.DATA_INIZIO.strftime('%Y-%m-%d'),
            'ultima_data': self.DATA_FINE.strftime('%Y-%m-%d')
        }
        with open(f'tickers/_indice.json', 'w') as jsonfile:
            json.dump(indice, jsonfile, indent=4)    
 
    def avvia_screener(self, append=False) -> None:
        tot_tickers = len(self.lista_tickers)

        for i in range(0, tot_tickers):
            try:
                nome_simbolo = self.lista_tickers.iloc[i]["Ticker"]
                print("\033[42m" + f'{i+1}/{tot_tickers}) Caricamento ticker {nome_simbolo}' + "\033[0m")
                ticker = pd.read_hdf(f'tickers/{nome_simbolo}.h5', 'ticker')
                if append:
                    scr = pd.read_hdf(f'screeners/{nome_simbolo}.h5', 'screener')
                    inizio = scr.index.max() - pd.Timedelta(days=365)
                    ticker_analisi = ticker[ticker.index >= inizio]
                    if scr.index.max() == ticker.index.max():
                        scr = scr.drop(scr.index[-1])
                    idx, X, Y, _ = self.modello.to_XY(ticker_analisi, bilanciamento=0)
                    print(f'Aggiornamento previsione {nome_simbolo}')
                    pred = self.modello.model.predict(X)
                    scr_temp = pd.DataFrame({'Previsione': pred.flatten().round(2), 'Reale': Y.flatten()}, index=idx)
                    scr_temp = scr_temp[scr_temp.index > scr.index.max()]
                    scr = pd.concat([scr, scr_temp], axis=0, ignore_index=False)
                    print(f"Aggiornamento file {nome_simbolo}.h5")
                    scr.to_hdf(f'screeners/{nome_simbolo}.h5', key='screener', mode='w')
                else:
                    idx, X, Y, _ = self.modello.to_XY(ticker, bilanciamento=0)
                    print(f'Previsione {nome_simbolo}')
                    pred = self.modello.model.predict(X)
                    scr = pd.DataFrame({'Previsione': pred.flatten().round(2), 'Reale': Y.flatten()}, index=idx)
                    scr = scr[scr.index >= self.DATA_INIZIO]
                    print(f"Salvataggio file {nome_simbolo}.h5")
                    scr.to_hdf(f'screeners/{nome_simbolo}.h5', key='screener', mode='w')
            except Exception as e:
                print(str(e))

    def ultimo_screener(self) -> (pd.Timestamp, pd.DataFrame):
        # Ottieni l'ultimo valore dell'indice di livello 0
        ultimo_indice = self.screener.index.get_level_values(0)[-1]
        # Usa .loc per selezionare tutte le righe con quell'indice di livello 0
        return ultimo_indice.date(), self.screener.loc[ultimo_indice]

    def reset_trading(self) -> None:
        self._data_corrente = self.DATA_INIZIO
        self._posizioni = []
        self._bilancio = self.BILANCIO_INIZIALE
        self.esito_trading = pd.DataFrame()
        self._bilancio_per_simbolo = self.BILANCIO_INIZIALE / self.N_SIMBOLI
        self._valore_posizioni = 0

    def avvia_trading(self) -> None:
        self.reset_trading()
        while self._data_corrente <= self.DATA_FINE:
            print("\r\033[31m" + f'Data: {self._data_corrente.date()}, Pos. aperte: {len(self._posizioni)}' + "\033[0m", end=' ')
            self._chiudi_posizioni()
            if self._data_corrente in self.screener.index.get_level_values(0):
                scr = self.screener.loc[(self._data_corrente, slice(None)), :]
                i = 0
                esito = True
                while (len(self._posizioni) < self.N_SIMBOLI) and (i < len(scr)) and esito == True:
                    simbolo = scr.index[i][1]
                    esito = self._apri_posizione(simbolo)
                    i += 1
            self._data_corrente += timedelta(days=1)
        
        self._chiudi_posizioni(forza_chiusura=True)

    def _chiudi_posizioni(self, forza_chiusura=False):
        posizioni_da_mantenere = []
        for pos in self._posizioni:
            df = pos.chiudi(self._data_corrente, self._bilancio, forza_chiusura=forza_chiusura)
            if df is not None or forza_chiusura:
                self.esito_trading = pd.concat([self.esito_trading, df], axis=0, ignore_index=True)
                df = df.iloc[0]
                self._valore_posizioni -= df['Valore_posizione']
                self._bilancio = df['Bilancio']
                pos_da_aprire = (self.N_SIMBOLI - len(self._posizioni))
                if pos_da_aprire == 0:
                    self._bilancio_per_simbolo = 0.
                else:
                    self._bilancio_per_simbolo = self._bilancio / pos_da_aprire
                print(f"VENDI {df['Simbolo']} n.{df['n_azioni']} azioni a {df['Prezzo_unitario']} € = {df['Prezzo_tot']} €, Esito: {df['Esito']}, Perc: {df['Perc']}, Bilancio: {round(self._bilancio, 2)} € + valore pos. aperte: {round(self._valore_posizioni, 2)} € = {round(self._bilancio + self._valore_posizioni, 2)} €, Bilancio per simbolo: {round(self._bilancio_per_simbolo, 2)}")
            else:
                posizioni_da_mantenere.append(pos)
        self._posizioni = posizioni_da_mantenere

    def _apri_posizione(self, simbolo):
        try:
            ticker = pd.read_hdf(f'tickers/{simbolo}.h5', 'ticker')
            ticker = ticker[ticker.index >= self._data_corrente]
            prezzo = ticker["Open"].iloc[0]
            n_azioni = self._bilancio_per_simbolo // prezzo
            if (prezzo < self._bilancio_per_simbolo):
                pos = Posizione(simbolo, ticker, self._data_corrente, n_azioni, self._bilancio, self.GIORNI_POS, self.SL, self.TP)
                self._posizioni.append(pos)
                self._valore_posizioni += pos.valore
                self.esito_trading = pd.concat([self.esito_trading, pos.to_df()], axis=0, ignore_index=True)
                self._bilancio = pos.bilancio
                pos_da_aprire = (self.N_SIMBOLI - len(self._posizioni))
                if pos_da_aprire == 0:
                    self._bilancio_per_simbolo = 0.
                else:
                    self._bilancio_per_simbolo = self._bilancio / pos_da_aprire
                print(f'COMPRA {simbolo} n.{int(pos.n_azioni)} azioni a {round(pos.prezzo_unitario, 2)} € = {round(pos.prezzo_tot, 2)} €,  Bilancio: {round(self._bilancio, 2)} € + valore pos. aperte: {round(self._valore_posizioni, 2)} € = {round(self._bilancio + self._valore_posizioni, 2)} €, Bilancio per simbolo: {round(self._bilancio_per_simbolo, 2)}')
                return True
            else:
                return False
        except Exception as e:
            print(f'Errore in apertura posizione: {str(e)}')
            return False

class Modello:
    def __init__(self):
        self.nome = 'default'
        self.timesteps = 120
        self.giorni_previsione = 1
        self.features_scala_prezzo = ["EMA_20", "EMA_50", "EMA_100"]
        self.features_da_scalare_singolarmente = ["Volume"]
        self.features_oscillatori = ["MACDh", "AROONOSC", "TRIX", "DM_OSC", "TSI", "KVO"]
        self.features_no_scala = []
        self.features_candele = []
        self.targets = ["Target"]
        self.n_features = 10
        self.n_targets = 1
        #self.f1_score = F1Score(num_classes=1, average='macro', threshold=0.5)
        self.model = load_model(f"{self.nome}/model.h5", custom_objects={'Addons>F1Score': self.f1_score})

    def _crea_modello(self):
        model = Sequential()
        
        model.add(Conv1D(filters=128, kernel_size=5, activation='relu', input_shape=(self.timesteps, self.n_features)))
        model.add(BatchNormalization())  
        model.add(LSTM(100, return_sequences=True))
        model.add(Dropout(0.5)) 
        model.add(LSTM(50))
        model.add(Dropout(0.5))
        model.add(Dense(100, activation='relu', kernel_regularizer=regularizers.l2(0.02)))
        model.add(Dropout(0.5))
        model.add(Dense(80, activation='relu', kernel_regularizer=regularizers.l2(0.02)))
        model.add(Dropout(0.5))
        model.add(BatchNormalization())   
        model.add(Dense(40, activation='relu', kernel_regularizer=regularizers.l2(0.02)))
        model.add(Dropout(0.5))
        model.add(BatchNormalization())   
        model.add(Dense(1, activation='sigmoid'))    
        
        model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[tfa.metrics.F1Score(num_classes=1, average='macro', threshold=soglia), Precision(), Recall()])
    
        return model

    def crea(self, nome_modello, timesteps, giorni_previsione, features, targets):
        self.nome = nome_modello
        self._percorso_file = f'{nome_modello}/impostazioni.json'

        try:
            os.makedirs(self.nome, exist_ok=True)
            # Crea un file di configurazione vuoto
            open(self._percorso_file, 'w').close()

            self.timesteps = timesteps # n. barre del periodo passato per la ricerca di pattern, inclusa ultima data disponibile
            self.giorni_previsione = giorni_previsione  # giorni futuri di cui effettuare la previsione
            self.features_scala_prezzo = [ft for ft in _features_scala_prezzo_tutte if ft in features]
            self.features_da_scalare_singolarmente = [ft for ft in _features_da_scalare_singolarmente_tutte if ft in features]
            self.features_oscillatori = [ft for ft in _features_oscillatori_tutte if ft in features]
            self.features_no_scala = [ft for ft in _features_no_scala_tutte if ft in features]
            self.features_candele = [ft for ft in _features_candele_tutte if ft in features]
            self.targets = targets

            self.impostazioni = {}
            self.impostazioni['timesteps'] = self.timesteps
            self.impostazioni['giorni_previsione'] = self.giorni_previsione
            self.impostazioni['features_scala_prezzo'] = self.features_scala_prezzo
            self.impostazioni['features_da_scalare_singolarmente'] = self.features_da_scalare_singolarmente
            self.impostazioni['features_oscillatori'] = self.features_oscillatori
            self.impostazioni['features_no_scala'] = self.features_no_scala
            self.impostazioni['features_candele'] = self.features_candele
            self.impostazioni['targets'] = self.targets
            # Salva impostazioni sul file
            self._salva_impostazioni()

            self.n_features = len(self.features_scala_prezzo) + len(self.features_da_scalare_singolarmente) + len(self.features_oscillatori) + len(self.features_no_scala) + len(self.features_candele) 
            self.n_targets = len(self.targets) 

            self.model = self._crea_modello()

        except OSError as errore:
            print(f"Errore durante la creazione della directory: {errore}")

    def carica(self, nome_modello):
        self.nome = nome_modello
        self._percorso_file = f'{nome_modello}/impostazioni.json'

        try:
            with open(self._percorso_file, "r") as file:
                # Carica il file o inizializza un dizionario vuoto se il file è vuoto
                self.impostazioni = json.load(file) if os.path.getsize(self._percorso_file) > 0 else {}
                # Carica impostazioni dal file se esistono, altrimenti carica i valori di default
                self.timesteps = self.impostazioni.get("timesteps", 120) # n. barre del periodo passato per la ricerca di pattern, inclusa ultima data disponibile
                self.giorni_previsione = self.impostazioni.get("giorni_previsione", 1)  # giorni futuri di cui effettuare la previsione
                self.features_scala_prezzo = self.impostazioni.get("features_scala_prezzo", ["EMA_20", "EMA_50", "EMA_100"])
                self.features_da_scalare_singolarmente = self.impostazioni.get("features_da_scalare_singolarmente", ["Volume"])
                self.features_oscillatori = self.impostazioni.get("features_oscillatori", ["MACDh", "AROONOSC", "TRIX", "DM_OSC", "TSI", "KVO"])
                self.features_no_scala = self.impostazioni.get("features_no_scala", [])
                self.features_candele = self.impostazioni.get("features_candele", [])
                self.targets = self.impostazioni.get("targets", ["Target"])
                # Salva impostazioni sul file
                self._salva_impostazioni()

                self.n_features = len(self.features_scala_prezzo) + len(self.features_da_scalare_singolarmente) + len(self.features_oscillatori) + len(self.features_no_scala) + len(self.features_candele) 
                self.n_targets = len(self.targets) 

                self.f1_score = F1Score(num_classes=1, average='macro', threshold=0.5)
                self.model = load_model(f"{self.nome}/model.h5", custom_objects={'Addons>F1Score': self.f1_score})

        except Exception as e:
            print(f"Errore durante il caricamento del file: {e}")

    def _salva_impostazioni(self):
        self.impostazioni = {
            "timesteps": self.timesteps,
            "giorni_previsione": self.giorni_previsione,
            "features_scala_prezzo": self.features_scala_prezzo,
            "features_da_scalare_singolarmente": self.features_da_scalare_singolarmente,
            "features_oscillatori": self.features_oscillatori,
            "features_no_scala": self.features_no_scala,
            "features_candele": self.features_candele,
            "targets": self.targets
        }
        with open(self._percorso_file, "w") as file:
            json.dump(self.impostazioni, file, indent=4)

    def salva(self):
        self._salva_impostazioni()
        self.model.save(f'{self.nome}/model.h5')

    def to_XY(self, dati_ticker, bilanciamento=0):
        scalers_prezzo = []
        scaler_meno_piu = MinMaxScaler(feature_range=(-1, 1))
        scaler_standard = MinMaxScaler()

        new_dates = pd.bdate_range(start=dati_ticker.index[-1] + pd.Timedelta(days=1), periods=self.giorni_previsione)
        df_new = pd.DataFrame(index=new_dates)
        dati_ticker = pd.concat([dati_ticker, df_new])

        ft_prezzo = dati_ticker[self.features_scala_prezzo]
        ft_standard = dati_ticker[self.features_da_scalare_singolarmente]
        ft_meno_piu = dati_ticker[self.features_oscillatori]
        ft_no_scala = dati_ticker[self.features_no_scala]
        ft_candele = dati_ticker[self.features_candele] 

        targets = dati_ticker[self.targets]

        i_tot = len(dati_ticker) - self.giorni_previsione

        tot_elementi = i_tot - (self.timesteps-1)    
        
        X_prezzo = X_standard = X_meno_piu = X_no_scala = X_candele = None
        if len(self.features_scala_prezzo) > 0:
            tot_col_prezzo_x = len(ft_prezzo.columns)
            X_prezzo = np.zeros((tot_elementi, self.timesteps, tot_col_prezzo_x))
        if len(self.features_da_scalare_singolarmente) > 0:
            tot_col_standard_x = len(ft_standard.columns)
            X_standard = np.zeros((tot_elementi, self.timesteps, tot_col_standard_x))
        if len(self.features_oscillatori) > 0:
            tot_col_meno_piu_x = len(ft_meno_piu.columns)
            X_meno_piu = np.zeros((tot_elementi, self.timesteps, tot_col_meno_piu_x))
        if len(self.features_no_scala) > 0:
            tot_col_no_scala_x = len(ft_no_scala.columns)
            X_no_scala = np.zeros((tot_elementi, self.timesteps, tot_col_no_scala_x))
        if len(self.features_candele) > 0:
            tot_col_candele_x = len(ft_candele.columns)
            X_candele = np.zeros((tot_elementi, self.timesteps, tot_col_candele_x))
        if len(self.targets) > 0:
            #tot_col_targets_y = len(targets.columns)
            #Y = np.zeros((tot_elementi, giorni_previsione, tot_col_targets_y)) # togliere se classificazione binaria
            Y = np.zeros(tot_elementi) #solo per classificazione binaria
        
        for i in range(self.timesteps - 1, i_tot):
            if len(self.features_scala_prezzo) > 0:
                arr_x = np.array(ft_prezzo.iloc[i - (self.timesteps - 1):i + 1])
                arr_res = arr_x.reshape(-1, 1)
                scaler_prezzo = MinMaxScaler()
                scaler_prezzo.fit(arr_res)
                arr_sc = scaler_prezzo.transform(arr_res).reshape(self.timesteps, tot_col_prezzo_x)
                X_prezzo[i - (self.timesteps - 1)] = arr_sc
                scalers_prezzo.append(scaler_prezzo)

            if len(self.features_da_scalare_singolarmente) > 0:
                arr_x = np.array(ft_standard.iloc[i - (self.timesteps - 1):i + 1])
                arr_sc = scaler_standard.fit_transform(arr_x)   
                X_standard[i - (self.timesteps - 1)] = arr_sc

            if len(self.features_oscillatori) > 0:
                arr_x = np.array(ft_meno_piu.iloc[i - (self.timesteps - 1):i + 1])
                arr_sc = scaler_meno_piu.fit_transform(arr_x)   
                X_meno_piu[i - (self.timesteps - 1)] = arr_sc

            if len(self.features_no_scala) > 0:
                arr_x = np.array(ft_no_scala.iloc[i - (self.timesteps - 1):i + 1])
                X_no_scala[i - (self.timesteps - 1)] = arr_x

            if len(self.features_candele) > 0:
                arr_x = np.array(ft_candele.iloc[i - (self.timesteps - 1):i + 1])
                arr_sc = scaler_meno_piu.fit_transform(arr_x) 
                X_candele[i - (self.timesteps - 1)] = arr_sc

            if len(self.targets) > 0:
                # arr_y = np.array(targets.iloc[i + 1:i + 1 + giorni_previsione]) # togliere in caso di classificazione binaria
                # arr_res = arr_y.reshape(-1, 1) # togliere in caso di classificazione binaria
                # arr_sc = scaler_prezzo.transform(arr_res).reshape(giorni_previsione, tot_col_targets_y) # togliere in caso di classificazione binaria
                # Y[i - (timesteps - 1)] = arr_sc  # togliere in caso di classificazione binaria
                Y[i - (self.timesteps - 1)] = np.array(targets.iloc[i]) #solo per classificazione binaria

        X_list = [x for x in [X_prezzo, X_standard, X_meno_piu, X_no_scala, X_candele] if x is not None and x.size > 0]
        X = np.concatenate(X_list, axis=2) if X_list else np.array([])
        idx = dati_ticker.index[self.timesteps - 1:i_tot]
    
        if bilanciamento > 0:
            rus = RandomUnderSampler(sampling_strategy=bilanciamento)
            dim1 = X.shape[1]
            dim2 = X.shape[2]
            X = X.reshape(-1, dim1 * dim2)
            X, Y = rus.fit_resample(X, Y)
            X = X.reshape(-1, dim1, dim2)

        Y = Y.reshape(-1, 1)
        return idx, X, Y, scalers_prezzo


/usr/local/lib/python3.9/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
/usr/local/lib/python3.9/dist-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.12.0 and strictly below 2.15.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.9.2 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're usin

In [2]:

if __name__ == "__main__":
    TP = 1
    SL = 0.01
    PROBABILITA_PER_ACQUISTO = 0.5
    BILANCIO_INIZIALE = 1000
    GIORNI_MAX_POSIZIONE = 20
    N_SIMBOLI = 10

    borsa = Borsa(N_SIMBOLI, BILANCIO_INIZIALE, PROBABILITA_PER_ACQUISTO, SL, TP, GIORNI_MAX_POSIZIONE)


In [3]:
data, scr = borsa.ultimo_screener()


In [ ]:
print(data)
scr.sample(frac=1).head(5)

2023-11-16


,Previsione,Reale
Ticker,,
RYAAY,0.61,0.0
IT,0.61,0.0
MSI,0.61,0.0
BBSI,0.61,0.0
CBT,0.60,0.0


In [5]:
#borsa.avvia_trading()
#borsa.esito_trading.to_excel('esito.xlsx')